In [49]:
# This notebook contains first of all stuff to collect German Bundesland Data from differnt sources
# And also saves it to the github (also cloned or forked versions)
# Skript for italy I will also provide but I have to copy it out. 

import geopandas as gpd
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta

from scipy.integrate import solve_ivp
from lmfit import Model

from zipfile import  ZipFile
import requests
from io import BytesIO

# load cumulative case data from wikipedia (Robert Koch based)
# change format and correct stuff 
data_laender_wiki=pd.read_html('https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland')
data_laender=pd.DataFrame(data_laender_wiki[19])
data_laender=data_laender.dropna().reset_index()

data_laender1=pd.DataFrame(data_laender_wiki[2])
data_laender1=data_laender1.dropna().reset_index().iloc[:,9:-1]
data_laender=data_laender.join(data_laender1)




date0=datetime(2020,2,22)# needs to be 2 less because of freaking column indexing
dates=[date0+timedelta(n) for n in range(len(data_laender.T))]

data_laender.columns=dates
bundeslaender=data_laender.iloc[0:16,1]
data_laender=data_laender.iloc[0:-2,2:]
columns=data_laender.columns
data_laender.replace(to_replace='—',value=0.0,inplace=True)
data_laender=pd.DataFrame(data_laender.values.astype(np.float64)/1.0)
data_laender['Bundesland']=bundeslaender
data_laender.set_index('Bundesland',inplace=True)
data_laender.columns=columns

#correct 1.000 = 1000.0 artefacts
data_laender[data_laender.round()!=data_laender]=data_laender[data_laender.round()!=data_laender]*1000.0


# This here is about data about the Bundesländer, I include only population related things
data_laender2=pd.read_html('https://de.wikipedia.org/wiki/Land_(Deutschland)')
data_laender2=data_laender2[0]
#include iso3166_2 for merge with geodata
data_laender2['iso_3166_2']='DE-'+data_laender2.iloc[:,2]

data_laender2=data_laender2.set_index('Land').iloc[:-1,7:]

data_laender2.iloc[:,0]=np.where(data_laender2.iloc[:,0]<100,data_laender2.iloc[:,0]*1000,data_laender2.iloc[:,0])
data_laender2.iloc[:,1]=data_laender2.iloc[:,1]/1000.0
data_laender2.iloc[:,2]=data_laender2.iloc[:,2]*1.0
data_laender2.iloc[:,3]=data_laender2.iloc[:,3]*0.1
data_laender2.drop('Sprachen',axis=1,inplace=True)

#fix strange columns due to wikipedia
data_laender2.columns=['Flaeche_km2','Einwohner_Mio','Einwohner_per_km2','Auslaender_proc','iso_3166_2']

#check if all columns are float for json
#data_laender2.info()

data_laender=data_laender.join(data_laender2)

# display and check all numeric is float64 (lmfit is otherwise in trouble with storing sav files == json)
#data_laender.info()

#save to the repository
last_date=str(columns[-1].date().isoformat())
data_laender.to_csv('./Deutschland_Bundeslander_cumdata_pandas_until_'+last_date+'.csv');
data_laender.to_csv('./Deutschland_Bundeslander_cumdata_pandas.csv');

#now get the geoinformation data needed for geopandas commented out just activate if you need

# also look at https://www.naturalearthdata.com/downloads/
# https://stackoverflow.com/questions/18885175/read-a-zipped-file-as-a-pandas-dataframe
# https://stackoverflow.com/questions/11914472/stringio-in-python3
#https://stackoverflow.com/questions/23419322/download-a-zip-file-and-extract-it-in-memory-using-python3



#r = requests.get('https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_1_states_provinces.zip')
#zip_regions=ZipFile(BytesIO(r.content))
#zip_regions.extractall('./natural_earth_admin1_large')

# load map for geopandas (large or high quality needed, medium = Canada Us)
gdf = gpd.read_file('./ne_10m_admin_1_states_provinces/ne_10m_admin_1_states_provinces.shp')
gdf.set_index(['iso_a2','name_vi'],inplace=True)
gdf.sort_values(['iso_a2','name_vi'],inplace=True)
gdf
map_germany=gdf.loc[('DE',slice(None))]

# now change index of data_laender to iso_3166_2 and join on map_germany

map_germany.set_index('iso_3166_2',inplace=True)
map_germany=map_germany.join(data_laender.set_index('iso_3166_2'))
map_germany=map_germany.loc[:,'geometry':]

#map_germany.info()


#write to file. Does not work yet. I dont know why
#map_germany.to_file('./Deutschland_Bundeslander_cumdata_geopandas_until_'+last_date+'.shp')


In [50]:
# below preliminary code for optimization. Note:  Data typically is selected so that only late data is included or weighted more
# I am not sure yet what is the appropriate way, but for example italian data looks very nice (for some) when fitted with all data,
# but it predicts very low R0 values, were most of the people in the field get values around 2ish I would say.
# Last 7 data selected and fitted gives for example R0=2 but confidence for beta and gamma is worse, an so are the confidence bands

# This code will end up being a script to fetch data from wikipedia for cum cases and for population data
# And I will include some script for loading map data for the use in geopandas

,2020-02-24 00:00:00,2020-02-25 00:00:00,2020-02-26 00:00:00,2020-02-27 00:00:00,2020-02-28 00:00:00,2020-02-29 00:00:00,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00,2020-03-04 00:00:00,...,2020-03-15 00:00:00,2020-03-16 00:00:00,2020-03-17 00:00:00,2020-03-18 00:00:00,2020-03-19 00:00:00,Flaeche_km2,Einwohner_Mio,Einwohner_per_km2,Auslaender_proc,iso_3166_2
Bundesland,,,,,,,,,,,,,,,,,,,,,
Baden-Württemberg,0.0,1.0,3.0,6.0,10.0,14.0,15.0,19.0,26.0,44.0,...,827.0,1105.0,1479.0,1609.0,2155.0,35751.0,10.880,304.0,14.2,DE-BW
Bayern,14.0,14.0,14.0,14.0,15.0,15.0,19.0,25.0,35.0,48.0,...,886.0,1067.0,1109.0,1243.0,1692.0,70550.0,12.844,182.0,12.3,DE-BY
Berlin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,6.0,...,265.0,300.0,345.0,391.0,573.0,892.0,3.520,3948.0,16.3,DE-BE
Brandenburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,84.0,94.0,73.0,92.0,134.0,29654.0,2.485,84.0,3.6,DE-BB
Bremen,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,...,53.0,56.0,57.0,69.0,80.0,420.0,0.671,1599.0,15.9,DE-HB
Hamburg,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,...,162.0,260.0,310.0,358.0,432.0,755.0,1.787,2366.0,15.4,DE-HH
Hessen,2.0,2.0,2.0,2.0,2.0,5.0,10.0,12.0,12.0,12.0,...,286.0,342.0,373.0,432.0,682.0,21115.0,6.176,293.0,15.1,DE-HE
Mecklenburg-Vorpommern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,50.0,51.0,45.0,56.0,98.0,23212.0,1.612,69.0,4.0,DE-MV
Niedersachsen,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,4.0,...,287.0,391.0,325.0,478.0,669.0,47593.0,7.927,167.0,8.4,DE-NI


In [142]:
from scipy.optimize import minimize
from lmfit import Model



# Make an objective function for optimization of italian data to SIR model
def SIR_model_without_R(t, y,beta,gamma,N):
    S=y[0]
    I=y[1]
    dSdt=-beta*I*S/N
    dIdt=beta*I*S/N-gamma*I    
    return [dSdt,dIdt]

def fit_function(t,beta,gamma,S0,I0,N):
    sol= solve_ivp(lambda t,y: SIR_model_without_R(t,y,beta,gamma,N), 
                   [0, max(t)],(S0,I0),teval=np.arange(0,max(t),1e-3),
                   dense_output=True,rtol=1e-10,atol=1e-20)
    y=sol.sol(t)
    return np.cumsum(y[1])

def model_function(t,beta,gamma,S0,I0,N):

    sol= solve_ivp(lambda t,y: SIR_model_without_R(t,y,beta,gamma,N), 
                   [0, max(t)],(S0,I0),teval=np.arange(0,max(t),1e-3),
                   dense_output=True,rtol=1e-10,atol=1e-20)
    y=sol.sol(t)
    return y


fit_Data=data_laender.copy()

cum_model = Model(fit_function)
cum_model.set_param_hint('beta',value=6,min=0,vary=True)
cum_model.set_param_hint('gamma',value=6,min=0,vary=True)
cum_model.set_param_hint('S0',min=0,vary=False)
cum_model.set_param_hint('I0',min=0,vary=False)
cum_model.set_param_hint('N',min=0,vary=False)
params1 = cum_model.make_params()

complete_model = Model(model_function)
complete_model.set_param_hint('beta',value=6,min=0,vary=True)
complete_model.set_param_hint('gamma',value=5.8,min=0,vary=True)
complete_model.set_param_hint('S0',min=0,vary=False)
complete_model.set_param_hint('I0',min=0,vary=False)
complete_model.set_param_hint('N',min=0,vary=False)
params2 = complete_model.make_params()


In [145]:
from lmfit.model import save_modelresult
from lmfit.model import load_modelresult
from pathlib import Path
    

def fit_model(gmodel,gmodel_2,Data,t,S0,I0,N,title1,title2,ax):
    #print(t,Data,N,Data[0])
    result=gmodel.fit(Data,t=t,S0=N,I0=Data[0],N=N)
    #print(result.fit_report())
    tnew=np.arange(len(Data)*3)
    best_fit=gmodel.eval(t=tnew,beta=result.values['beta'],gamma=result.values['gamma'],S0=N,I0=Data[0],N=N)
    
    
    del_fit = result.eval_uncertainty(t=tnew)
    upper_band=best_fit+del_fit
    lower_band=best_fit-del_fit
    ax[0].plot(t, Data, 'bo',label='Data')
    ax[0].plot(tnew, best_fit, 'r-', label='best fit SIR model')
    ax[0].plot(tnew,upper_band, 'k-', label='conf bands')
    ax[0].plot(tnew,lower_band,'k-')
    ax[0].legend(loc='best',fontsize=18)
    ax[0].set_title(title1,fontsize=20)
    ax[0].set_xlabel('time/days',fontsize=18)
    ax[0].set_ylabel('Total infected persons',fontsize=18)
    plt.setp(ax[0].get_yticklabels(),fontsize=16)
    plt.setp(ax[0].get_xticklabels(),fontsize=16)
        
    tnew2=np.arange(0,len(Data)*3,0.01)    
    best_SI=gmodel_2.eval(t=tnew2,beta=result.values['beta'],gamma=result.values['gamma'],S0=N,I0=Data[0],N=N)
    ax[1].plot(tnew2,best_SI[1].T/N*100,'k-')
    ax[2].plot(tnew2,best_SI[0].T/N*100)
    ax[1].set_xlabel('time/days',fontsize=18)
    ax[1].set_ylabel('I (infected persons) / %',fontsize=18)
    ax[1].set_title(title2,fontsize=20)    
    ax[2].set_ylabel('S (Not infected persons) / %',fontsize=18)
    plt.setp(ax[1].get_yticklabels(),fontsize=16)
    plt.setp(ax[1].get_xticklabels(),fontsize=16)
    plt.setp(ax[2].get_yticklabels(),fontsize=16)
    
    Data_plot=pd.DataFrame({'t':t,'Data':Data})
    best_fit=pd.DataFrame({'t':tnew,'best_fit':best_fit})
    upper=pd.DataFrame({'t':tnew,'upper_band':upper_band})
    lower=pd.DataFrame({'t':tnew,'lower_band':lower_band})
    best_fit_S=pd.DataFrame({'t':tnew2,'best_fit_S':best_SI[0].T})
    best_fit_I=pd.DataFrame({'t':tnew2,'best_fit_I':best_SI[1].T})
    
    return result,Data_plot,best_fit,upper,lower,best_fit_S,best_fit_I



fit_Data=data_laender.copy()


fig,ax=plt.subplots(1,2,figsize=[20,10])    
ax2=ax[1].twinx()
ax=[ax[0],ax[1],ax2]
print(len(fit_Data),end=' ')

fit_result_Data=pd.DataFrame({'lmfit_result_files':[],'Data_plots':[],'best_fits':[],
                       'upper':[],'lower':[],'best_fit_S':[],'best_fit_I':[]})
try:
    fit_result_Data=pd.read_json('Germany_SIR_fit.json')
except:
    pass
for n in range(len(fit_Data)):
    print(n,end='')
    if (n<len(fit_result_Data)):
        if fit_result_Data.loc[n][0]!=None:
            print('s',end=' ')
            continue
    Data=fit_Data.iloc[n,:-3].values
    N=fit_Data.iloc[n,-2]
    if max(Data)>0:
        while Data[0]==0:
            Data=Data[1:]
    else:
        fit_result_Data.loc[n]=[None,None,None,None,None,None,None]
        continue
    t=np.arange(len(Data))
    S0=N
    I0=Data[0]
    try:
        result,Data_plot,best_fit,upper,lower,best_fit_S,best_fit_I=fit_model(cum_model,complete_model,Data,t,S0,I0,N,f'Cumulative infections in {bundeslaender[n]}',f'S and I {bundeslaender[n]}',ax);
        print('x',end='')
        #save result to separate file
        result_file=f'Deutschland_SIR_fitresult_{n:003}_{bundeslaender[n]}.sav'
        save_modelresult(result,result_file)

        #save figure to png
    
        if Path(f'Deutschland_SIR_fitresult_{n:003}_{bundeslaender[n]}.png').is_file():
            print ('x',end=' ')
        else:
            fig.savefig(f'Deutschland_SIR_fitresult_{n:003}_{bundeslaender[n]}.png', bbox_inches='tight', dpi=400);
            print ('y',end=' ')
        fit_result_Data.loc[n]=[result_file,Data_plot,best_fit,upper,lower,best_fit_S,best_fit_I]
        
    except:
        fit_result_Data.loc[n]=[None,None,None,None,None,None,None]
    ax[0].clear();
    ax[1].clear();
    ax[2].clear(); 

fig.clf()
fit_result_Data.to_json('Germany_SIR_fit.json')

16 0xx 1xx 2xx 3xx 4xy 5xy 6xy 7xy 8xy 9xy 10xy 11xy 12xy 13xy 14xy 15xy 

<Figure size 1440x720 with 0 Axes>